# 2.3 Blank Nodes

Blank nodes enable us to represent entities that themselves have properties or relate to other nodes, without giving the node itself an IRI.

This is particularly relevant if the node itself is uninteresting or unknown but its relationships to the rest of the graph are, in which case a composite object that provides several details is more helpful.

Blank nodes are created without specifying an IRI - RDFox will generate a random identifier for this node in order to keep track of it. This can present challenges as the identifier is not generated deterministically and will be different every time the data is imported.

While blank nodes can be incredible useful, they can be difficult to work with due to this inconsistent anonymized identity which is why, in practice, we highly encourage using them with SKOLEM.

## SKOLEM

SKOLEM generates a blank node with a deterministic IRI from a series of input parameters of both variables and literals.

This means the new node retains a referenceable, reversible, meaningful IRI that can be based solely on selected properties.

### SKOLEM syntax

SKOLEM("literalOr", ?variable, ..., ?result)

Syntactically, the last argument given to SKOLEM is the variable that will be bound to the generated value. All preceding arguments contribute to the generation.

Arguments can be either a variable or string.

## Example

This example examines sensor readings of liquid baths in a manufacturing process, where each bath is monitored over time by two sensors.

We'll create a blank node with SKOLEM that represents a summary of the bath states as they change over time.

In [19]:
bn_data = """
@prefix : <https://rdfox.com/example#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:readingA1 a :SensorReading ;
    :fromSensor :sensorA ;
    :hasTemperature 18 ;
    :hasTimeStamp "2024-12-25T12:00:00"^^xsd:dateTime .

:readingB1 a :SensorReading ;
    :fromSensor :sensorB ;
    :hasTemperature 20 ;
    :hasTimeStamp "2024-12-25T12:00:00"^^xsd:dateTime .

:readingA2 a :SensorReading ;
    :fromSensor :sensorA ;
    :hasTemperature 21 ;
    :hasTimeStamp "2024-12-25T12:01:00"^^xsd:dateTime .

:readingB2 a :SensorReading ;
    :fromSensor :sensorB ;
    :hasTemperature 23 ;
    :hasTimeStamp "2024-12-25T12:01:00"^^xsd:dateTime .

"""

In [20]:
bn_rules = """

[?bathReading, a, :BathSummary],
[?bathReading, :hasTimeStamp, ?time],
[?bathReading, :hasTemperatureEstimate, ?tempEstimate] :-
    [?reading1, :fromSensor, :sensorA],
    [?reading2, :fromSensor, :sensorB],
    [?reading1, :hasTimeStamp, ?time],
    [?reading2, :hasTimeStamp, ?time],
    [?reading1, :hasTemperature, ?temp1],
    [?reading2, :hasTemperature, ?temp2],
    BIND((?temp1 + ?temp2)/2 AS ?tempEstimate),
    SKOLEM("SensorGroup", ?time, ?bathReading) .

"""

In [21]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=bn_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=bn_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Get and issue select query
with open("../queries/2_3-BlankNodesQuery.rq", "r") as file:
    bn_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": bn_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Average Bath Temperature Estimate Over Time ===')
print(response.text)


=== Average Bath Temperature Estimate Over Time ===
?bathReadingEncoded	?time	?temperatureEstimate
_:_.05U2Vuc29yR3JvdXAA.08AP7cDxc6AADoBwAAAAAAAACAAAAMGQwA	"2024-12-25T12:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	19.0
_:_.05U2Vuc29yR3JvdXAA.08YOjdDxc6AADoBwAAAAAAAACAAAAMGQwB	"2024-12-25T12:01:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	22.0



### Visualise the results

Open this query in the [RDFox Explorer](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FbathReadingEncoded%20%3Ftime%20%3FtemperatureEstimate%20%0AWHERE%20%7B%0A%20%20%20%20%3FbathReadingEncoded%20a%20%3ABathSummary%20%3B%0A%20%20%20%20%20%20%20%20%3AhasTimeStamp%20%3Ftime%20%3B%0A%20%20%20%20%20%20%20%20%3AhasTemperatureEstimate%20%3FtemperatureEstimate%20.%0A%7D%20ORDER%20BY%20ASC%28%3Ftime%29).

## Reversing SKOLEM

SKOLEM is a reversible process. Given a SKOLEMized node IRI, we can determine the resources that were used to create it.

Here is a rule that unpicks the SKOLEM node we just created.

In [17]:
sk_rules = """
    [?sting, :derivedFrom, ?bathReading],
    [?time, :derivedFrom, ?bathReading] :-
        [?bathReading, a, :BathSummary],
        SKOLEM(?sting, ?time, ?bathReading) .
"""

sk_sparql = """

SELECT ?SKOLEMNode ?thing
WHERE {
    ?thing :derivedFrom ?SKOLEMNode .
} ORDER BY ASC(?SKOLEMNode)

"""

# Add rule
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=sk_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": sk_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n============ SKOLEM reversed ============')
print(response.text)


============ SKOLEM reversed ============
?SKOLEMNode	?thing
_:_.05U2Vuc29yR3JvdXAA.08AP7cDxc6AADoBwAAAAAAAACAAAAMGQwA	"2024-12-25T12:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>
_:_.05U2Vuc29yR3JvdXAA.08AP7cDxc6AADoBwAAAAAAAACAAAAMGQwA	"SensorGroup"
_:_.05U2Vuc29yR3JvdXAA.08YOjdDxc6AADoBwAAAAAAAACAAAAMGQwB	"2024-12-25T12:01:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>
_:_.05U2Vuc29yR3JvdXAA.08YOjdDxc6AADoBwAAAAAAAACAAAAMGQwB	"SensorGroup"



In [18]:
# Clear rules from the data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?rules=true")
assert_response_ok(clear_response, "Failed to clear rules from data store.")

### MD5 Encoding

Sometimes, the MD5 hash generator function is instead of SKOLEM to create an ID for a node.

MD5 performs more efficiently than SKOLEM but has some drawbacks.

Primarily, it is non-reversible and it is not strictly 1-1, so it is possible, albeit unlikely, to have two different inputs create the same output.

Secondly, as a function, not a tuple table, the out of MD5 must be bound to a new variable, and it's input and output must be strings.

`BIND ( MD5("stringIn") AS ?stringOut )`

## Where are blank nodes relevant?

Blank nodes are incredibly versatile, allowing abstract representations of data to be added directly to the graph.

As such, they can be used in almost any application, including:

### Finance

To generate objects that capture sprawling details such as in fraud chains, suspicious activity, or regulation breaches etc.

### Iot & On-device

To structure event data, surface high level system properties, catalog time-series events, etc.

### Data Management

To create useful structures from complex data, generate abstractions, etc.

## Exercise

Complete the rule `blankNodesRules.dlog` in the `rules` folder to extend the function of the example to incorporate multiple baths, each with two sensors, as they change over time.

### Hits & helpful resources

[SKOLEM in RDFox](https://docs.oxfordsemantic.tech/tuple-tables.html#skolem)


In [22]:
bn_sparql = """

SELECT ?bath ?time ?temperatureEstimate
WHERE {
    ?bathReading a :BathSummary ;
        :locatedIn ?bath ;
        :hasTimeStamp ?time ;
        :hasTemperatureEstimate ?temperatureEstimate .
} ORDER BY ASC(?bath) ASC(?time)

"""

Here is a representative sample of the data in `2_3-BlankNodesData.ttl`.

In [23]:
sample_data = """
@prefix : <https://rdfox.com/example#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:bathAlpha a :Bath .

:sensorA a :Sensor ;
    :locatedIn :bathAlpha .

:sensorB a :Sensor ;
    :locatedIn :bathAlpha.

:readingA001 a :SensorReading ;
    :fromSensor :sensorA ;
    :hasTemperature 18 ;
    :hasTimeStamp "2024-12-25T12:00:00"^^xsd:dateTime .

:readingB001 a :SensorReading ;
    :fromSensor :sensorB ;
    :hasTemperature 20 ;
    :hasTimeStamp "2024-12-25T12:00:00"^^xsd:dateTime .

"""

### Check your work

Run the query below to verify the results.

In [25]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/2_3-BlankNodesData.ttl", "r") as file:
    bn_data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=bn_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/2_3-BlankNodesRules.dlog", "r") as file:
    bn_rules = file.read()
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=bn_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": bn_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Bath temperature over time ===')
print(response.text)


=== Bath temperature over time ===
?bath	?time	?temperatureEstimate
<https://rdfox.com/example#bathAlpha>	"2025-12-25T00:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	16.0
<https://rdfox.com/example#bathAlpha>	"2025-12-25T02:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	16.5
<https://rdfox.com/example#bathAlpha>	"2025-12-25T04:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	17.5
<https://rdfox.com/example#bathAlpha>	"2025-12-25T06:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	19.0
<https://rdfox.com/example#bathAlpha>	"2025-12-25T08:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	20.0
<https://rdfox.com/example#bathAlpha>	"2025-12-25T10:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	22.0
<https://rdfox.com/example#bathAlpha>	"2025-12-25T12:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	23.5
<https://rdfox.com/example#bathAlpha>	"2025-12-25T14:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>	23.5
<https://rdfox.com/example#bathAlpha>	"2025-12-25T1

## You should see...

=== Bath temperature over time ===
|?bath|?time|?temperatureEstimate|
|-----------|-------------|-------------|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T00:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	16.0|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T02:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	16.5|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T04:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	17.5|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T06:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	19.0|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T08:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	20.0|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T10:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	22.0|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T12:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	23.5|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T14:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	23.5|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T16:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	24.0|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T18:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	23.5|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T20:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	24.5|
|<https://rdfox.com/example#bathAlpha>|	"2025-12-25T22:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	24.0|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T00:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	16.0|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T02:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	16.0|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T04:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	16.5|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T06:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	16.5|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T08:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	17.5|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T10:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	19.5|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T12:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	20.5|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T14:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	21.5|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T16:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	22.5|
|<https://rdfox.com/example#bathBeta>|	"2025-12-25T18:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	23.5|
|...|||
|<https://rdfox.com/example#bathGamma>|	"2025-12-25T18:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	24.5|
|<https://rdfox.com/example#bathGamma>|	"2025-12-25T20:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	26.5|
|<https://rdfox.com/example#bathGamma>|	"2025-12-25T22:00:00"^^<http://www.w3.org/2001/XMLSchema#dateTime>|	26.0|